# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 12:00PM,241411,12,U0959,"Uniderm USA, Inc",Released
1,Jul 8 2022 11:55AM,241410,19,ACG-2102490551,ACG North America LLC,Released
2,Jul 8 2022 11:48AM,241409,10,0085842765,ISDIN Corporation,Released
3,Jul 8 2022 11:48AM,241409,10,0085842786,ISDIN Corporation,Released
4,Jul 8 2022 11:48AM,241409,10,0085842785,ISDIN Corporation,Released
5,Jul 8 2022 11:48AM,241409,10,0085842784,ISDIN Corporation,Released
6,Jul 8 2022 11:48AM,241409,10,0085842800,ISDIN Corporation,Released
7,Jul 8 2022 11:48AM,241409,10,0085842808,ISDIN Corporation,Released
8,Jul 8 2022 11:48AM,241409,10,0085842809,ISDIN Corporation,Released
9,Jul 8 2022 11:48AM,241409,10,0085842814,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,241407,Released,54
46,241408,Released,1
47,241409,Released,24
48,241410,Released,1
49,241411,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241407,NaN,NaN,54.0
241408,NaN,NaN,1.0
241409,NaN,NaN,24.0
241410,NaN,NaN,1.0
241411,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241319,32.0,9.0,2.0
241320,0.0,0.0,1.0
241326,0.0,0.0,1.0
241343,0.0,1.0,0.0
241345,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241319,32,9,2
241320,0,0,1
241326,0,0,1
241343,0,1,0
241345,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241319,32,9,2
1,241320,0,0,1
2,241326,0,0,1
3,241343,0,1,0
4,241345,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241319,32,9,2
1,241320,,,1
2,241326,,,1
3,241343,,1,
4,241345,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 12:00PM,241411,12,"Uniderm USA, Inc"
1,Jul 8 2022 11:55AM,241410,19,ACG North America LLC
2,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation
26,Jul 8 2022 11:48AM,241407,10,ISDIN Corporation
80,Jul 8 2022 11:44AM,241408,10,Livs Products
81,Jul 8 2022 11:37AM,241405,10,Livs Products
82,Jul 8 2022 11:33AM,241404,10,"Citieffe, Inc."
83,Jul 8 2022 11:24AM,241403,10,"Citieffe, Inc."
84,Jul 8 2022 11:22AM,241401,10,Emerginnova
86,Jul 8 2022 11:20AM,241400,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 12:00PM,241411,12,"Uniderm USA, Inc",,,1
1,Jul 8 2022 11:55AM,241410,19,ACG North America LLC,,,1
2,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,,,24
3,Jul 8 2022 11:48AM,241407,10,ISDIN Corporation,,,54
4,Jul 8 2022 11:44AM,241408,10,Livs Products,,,1
5,Jul 8 2022 11:37AM,241405,10,Livs Products,,,1
6,Jul 8 2022 11:33AM,241404,10,"Citieffe, Inc.",,,1
7,Jul 8 2022 11:24AM,241403,10,"Citieffe, Inc.",,,1
8,Jul 8 2022 11:22AM,241401,10,Emerginnova,,,2
9,Jul 8 2022 11:20AM,241400,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:00PM,241411,12,"Uniderm USA, Inc",1,,
1,Jul 8 2022 11:55AM,241410,19,ACG North America LLC,1,,
2,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,24,,
3,Jul 8 2022 11:48AM,241407,10,ISDIN Corporation,54,,
4,Jul 8 2022 11:44AM,241408,10,Livs Products,1,,
5,Jul 8 2022 11:37AM,241405,10,Livs Products,1,,
6,Jul 8 2022 11:33AM,241404,10,"Citieffe, Inc.",1,,
7,Jul 8 2022 11:24AM,241403,10,"Citieffe, Inc.",1,,
8,Jul 8 2022 11:22AM,241401,10,Emerginnova,2,,
9,Jul 8 2022 11:20AM,241400,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:00PM,241411,12,"Uniderm USA, Inc",1,,
1,Jul 8 2022 11:55AM,241410,19,ACG North America LLC,1,,
2,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,24,,
3,Jul 8 2022 11:48AM,241407,10,ISDIN Corporation,54,,
4,Jul 8 2022 11:44AM,241408,10,Livs Products,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:00PM,241411,12,"Uniderm USA, Inc",1.0,NaN,NaN
1,Jul 8 2022 11:55AM,241410,19,ACG North America LLC,1.0,NaN,NaN
2,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,24.0,NaN,NaN
3,Jul 8 2022 11:48AM,241407,10,ISDIN Corporation,54.0,NaN,NaN
4,Jul 8 2022 11:44AM,241408,10,Livs Products,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 12:00PM,241411,12,"Uniderm USA, Inc",1.0,0.0,0.0
1,Jul 8 2022 11:55AM,241410,19,ACG North America LLC,1.0,0.0,0.0
2,Jul 8 2022 11:48AM,241409,10,ISDIN Corporation,24.0,0.0,0.0
3,Jul 8 2022 11:48AM,241407,10,ISDIN Corporation,54.0,0.0,0.0
4,Jul 8 2022 11:44AM,241408,10,Livs Products,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6034654,261.0,1.0,0.0
102,482646,2.0,0.0,0.0
12,482791,11.0,0.0,0.0
15,482757,161.0,18.0,0.0
16,241373,0.0,1.0,0.0
18,241351,0.0,1.0,0.0
19,724168,1.0,10.0,16.0
20,965482,24.0,16.0,32.0
21,965383,2.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6034654,261.0,1.0,0.0
1,102,482646,2.0,0.0,0.0
2,12,482791,11.0,0.0,0.0
3,15,482757,161.0,18.0,0.0
4,16,241373,0.0,1.0,0.0
5,18,241351,0.0,1.0,0.0
6,19,724168,1.0,10.0,16.0
7,20,965482,24.0,16.0,32.0
8,21,965383,2.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,261.0,1.0,0.0
1,102,2.0,0.0,0.0
2,12,11.0,0.0,0.0
3,15,161.0,18.0,0.0
4,16,0.0,1.0,0.0
5,18,0.0,1.0,0.0
6,19,1.0,10.0,16.0
7,20,24.0,16.0,32.0
8,21,2.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,261.0
1,102,Released,2.0
2,12,Released,11.0
3,15,Released,161.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,16.0,32.0,0.0
Executing,1.0,0.0,0.0,18.0,1.0,1.0,10.0,16.0,2.0
Released,261.0,2.0,11.0,161.0,0.0,0.0,1.0,24.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,16.0,32.0,0.0
1,Executing,1.0,0.0,0.0,18.0,1.0,1.0,10.0,16.0,2.0
2,Released,261.0,2.0,11.0,161.0,0.0,0.0,1.0,24.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,16.0,32.0,0.0
1,Executing,1.0,0.0,0.0,18.0,1.0,1.0,10.0,16.0,2.0
2,Released,261.0,2.0,11.0,161.0,0.0,0.0,1.0,24.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()